In [4]:
"""
The most atomic way to train and inference a GPT with DeepSeek Sparse Attention and RoPE.
This file is the complete algorithm with sparse attention and rotary embeddings.
Everything else is just efficiency.

Extensions to @karpathy's microgpt:
- DeepSeek Sparse Attention: attend to local window + top-k global tokens
- RoPE: rotary position embeddings instead of learned positional embeddings
"""

import os       # os.path.exists
import math     # math.log, math.exp, math.sin, math.cos
import random   # random.seed, random.choices, random.gauss, random.shuffle
random.seed(42) # Let there be order among chaos

# Let there be an input dataset `docs`: list[str] of documents (e.g. a dataset of names)
if not os.path.exists('input.txt'):
    import urllib.request
    names_url = 'https://raw.githubusercontent.com/karpathy/makemore/refs/heads/master/names.txt'
    urllib.request.urlretrieve(names_url, 'input.txt')
docs = [l.strip() for l in open('input.txt').read().strip().split('\n') if l.strip()] # list[str] of documents
random.shuffle(docs)
print(f"num docs: {len(docs)}")

# Let there be a Tokenizer to translate strings to discrete symbols and back
uchars = sorted(set(''.join(docs))) # unique characters in the dataset become token ids 0..n-1
BOS = len(uchars) # token id for the special Beginning of Sequence (BOS) token
vocab_size = len(uchars) + 1 # total number of unique tokens, +1 is for BOS
print(f"vocab size: {vocab_size}")

# Let there be Autograd, to recursively apply the chain rule through a computation graph
class Value:
    __slots__ = ('data', 'grad', '_children', '_local_grads') # Python optimization for memory usage

    def __init__(self, data, children=(), local_grads=()):
        self.data = data                # scalar value of this node calculated during forward pass
        self.grad = 0                   # derivative of the loss w.r.t. this node, calculated in backward pass
        self._children = children       # children of this node in the computation graph
        self._local_grads = local_grads # local derivative of this node w.r.t. its children

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        return Value(self.data + other.data, (self, other), (1, 1))

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        return Value(self.data * other.data, (self, other), (other.data, self.data))

    def __pow__(self, other): return Value(self.data**other, (self,), (other * self.data**(other-1),))
    def log(self): return Value(math.log(self.data), (self,), (1/self.data,))
    def exp(self): return Value(math.exp(self.data), (self,), (math.exp(self.data),))
    def relu(self): return Value(max(0, self.data), (self,), (float(self.data > 0),))
    def __neg__(self): return self * -1
    def __radd__(self, other): return self + other
    def __sub__(self, other): return self + (-other)
    def __rsub__(self, other): return other + (-self)
    def __rmul__(self, other): return self * other
    def __truediv__(self, other): return self * other**-1
    def __rtruediv__(self, other): return other * self**-1

    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._children:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        self.grad = 1
        for v in reversed(topo):
            for child, local_grad in zip(v._children, v._local_grads):
                child.grad += local_grad * v.grad


num docs: 32033
vocab size: 27


In [5]:

# Initialize the parameters, to store the knowledge of the model.
n_embd = 16         # embedding dimension
n_head = 4          # number of attention heads
n_layer = 1         # number of layers
block_size = 16     # maximum sequence length
head_dim = n_embd // n_head # dimension of each head
local_window = 4    # local attention window size (attend to nearest tokens)
top_k_global = 2    # number of top-k global tokens to attend to (sparse attention)
matrix = lambda nout, nin, std=0.08: [[Value(random.gauss(0, std)) for _ in range(nin)] for _ in range(nout)]
state_dict = {'wte': matrix(vocab_size, n_embd), 'lm_head': matrix(vocab_size, n_embd)}
# Note: RoPE replaces learned positional embeddings, so no 'wpe' here
for i in range(n_layer):
    state_dict[f'layer{i}.attn_wq'] = matrix(n_embd, n_embd)
    state_dict[f'layer{i}.attn_wk'] = matrix(n_embd, n_embd)
    state_dict[f'layer{i}.attn_wv'] = matrix(n_embd, n_embd)
    state_dict[f'layer{i}.attn_wo'] = matrix(n_embd, n_embd)
    state_dict[f'layer{i}.mlp_fc1'] = matrix(4 * n_embd, n_embd)
    state_dict[f'layer{i}.mlp_fc2'] = matrix(n_embd, 4 * n_embd)
params = [p for mat in state_dict.values() for row in mat for p in row] # flatten params into a single list[Value]
print(f"num params: {len(params)}")

# Let there be RoPE, the blessed rotary position embeddings that encode position through rotation
def apply_rope(vec, pos_id):
    """Apply Rotary Position Embeddings to a vector by rotating pairs of dimensions."""
    result = []
    for i in range(0, len(vec), 2):
        # Each pair of dimensions rotates by an angle determined by position and dimension index
        theta = pos_id / (10000 ** (i / len(vec)))
        cos_theta = Value(math.cos(theta))
        sin_theta = Value(math.sin(theta))
        # Rotation matrix: [cos -sin] [x0]
        #                  [sin  cos] [x1]
        x0, x1 = vec[i], vec[i+1]
        result.append(cos_theta * x0 - sin_theta * x1)
        result.append(sin_theta * x0 + cos_theta * x1)
    return result

# Define the model architecture: a stateless function mapping token sequence and parameters to logits over what comes next.
# Follow GPT-2, blessed among the GPTs, with extensions: RoPE for position, DSA for sparse attention
def linear(x, w):
    return [sum(wi * xi for wi, xi in zip(wo, x)) for wo in w]

def softmax(logits):
    max_val = max(val.data for val in logits)
    exps = [(val - max_val).exp() for val in logits]
    total = sum(exps)
    return [e / total for e in exps]

def rmsnorm(x):
    ms = sum(xi * xi for xi in x) / len(x)
    scale = (ms + 1e-5) ** -0.5
    return [xi * scale for xi in x]

def sparse_attention_mask(current_pos, seq_len):
    """
    DeepSeek Sparse Attention: determine which positions to attend to.
    Returns a list of positions to attend to (local window + will select top-k global later).
    """
    # Local window: attend to nearby tokens
    local_positions = [p for p in range(max(0, current_pos - local_window + 1), current_pos + 1)]
    # Global candidates: all other positions before local window
    global_candidates = [p for p in range(current_pos + 1) if p not in local_positions]
    return local_positions, global_candidates


num params: 3936


In [6]:

def gpt(token_id, pos_id, keys, values):
    tok_emb = state_dict['wte'][token_id] # token embedding
    x = tok_emb # no positional embedding added here; RoPE applies directly to Q and K
    x = rmsnorm(x)

    for li in range(n_layer):
        # 1) Multi-head sparse attention block with RoPE
        x_residual = x
        x = rmsnorm(x)
        q = linear(x, state_dict[f'layer{li}.attn_wq'])
        k = linear(x, state_dict[f'layer{li}.attn_wk'])
        v = linear(x, state_dict[f'layer{li}.attn_wv'])
        keys[li].append(k)
        values[li].append(v)
        seq_len = len(keys[li])
        
        x_attn = []
        for h in range(n_head):
            hs = h * head_dim
            # Extract head-specific query
            q_h = q[hs:hs+head_dim]
            # Apply RoPE to query
            q_h = apply_rope(q_h, pos_id)
            
            # DeepSeek Sparse Attention: determine which positions to attend to
            local_positions, global_candidates = sparse_attention_mask(pos_id, seq_len)
            
            # Compute attention scores for local positions (always attend)
            local_scores = []
            for t in local_positions:
                k_h = [keys[li][t][hs+j] for j in range(head_dim)]
                k_h = apply_rope(k_h, t) # Apply RoPE to key
                score = sum(q_h[j] * k_h[j] for j in range(head_dim)) / head_dim**0.5
                local_scores.append((t, score))
            
            # Compute attention scores for global candidates (sparse selection)
            global_scores = []
            for t in global_candidates:
                k_h = [keys[li][t][hs+j] for j in range(head_dim)]
                k_h = apply_rope(k_h, t) # Apply RoPE to key
                score = sum(q_h[j] * k_h[j] for j in range(head_dim)) / head_dim**0.5
                global_scores.append((t, score))
            
            # Select top-k global positions based on attention scores
            global_scores.sort(key=lambda x: x[1].data, reverse=True)
            top_k_positions = global_scores[:top_k_global] if len(global_scores) > 0 else []
            
            # Combine local and top-k global positions
            selected_positions = local_scores + top_k_positions
            selected_positions.sort(key=lambda x: x[0]) # sort by position for causality
            
            # Compute softmax over selected positions only (sparse attention)
            attn_logits = [score for _, score in selected_positions]
            attn_weights = softmax(attn_logits)
            
            # Weighted sum of values at selected positions
            head_out = []
            for j in range(head_dim):
                weighted_sum = Value(0)
                for idx, (t, _) in enumerate(selected_positions):
                    v_h_j = values[li][t][hs+j]
                    weighted_sum = weighted_sum + attn_weights[idx] * v_h_j
                head_out.append(weighted_sum)
            
            x_attn.extend(head_out)
        
        x = linear(x_attn, state_dict[f'layer{li}.attn_wo'])
        x = [a + b for a, b in zip(x, x_residual)]
        
        # 2) MLP block
        x_residual = x
        x = rmsnorm(x)
        x = linear(x, state_dict[f'layer{li}.mlp_fc1'])
        x = [xi.relu() for xi in x]
        x = linear(x, state_dict[f'layer{li}.mlp_fc2'])
        x = [a + b for a, b in zip(x, x_residual)]

    logits = linear(x, state_dict['lm_head'])
    return logits


In [7]:

# Let there be Adam, the blessed optimizer and its buffers
learning_rate, beta1, beta2, eps_adam = 0.01, 0.85, 0.99, 1e-8
m = [0.0] * len(params) # first moment buffer
v = [0.0] * len(params) # second moment buffer

# Repeat in sequence
num_steps = 1000 # number of training steps
for step in range(num_steps):

    # Take single document, tokenize it, surround it with BOS special token on both sides
    doc = docs[step % len(docs)]
    tokens = [BOS] + [uchars.index(ch) for ch in doc] + [BOS]
    n = min(block_size, len(tokens) - 1)

    # Forward the token sequence through the model, building up the computation graph all the way to the loss.
    keys, values = [[] for _ in range(n_layer)], [[] for _ in range(n_layer)]
    losses = []
    for pos_id in range(n):
        token_id, target_id = tokens[pos_id], tokens[pos_id + 1]
        logits = gpt(token_id, pos_id, keys, values)
        probs = softmax(logits)
        loss_t = -probs[target_id].log()
        losses.append(loss_t)
    loss = (1 / n) * sum(losses) # final average loss over the document sequence. May yours be low.

    # Backward the loss, calculating the gradients with respect to all model parameters.
    loss.backward()

    # Adam optimizer update: update the model parameters based on the corresponding gradients.
    lr_t = learning_rate * (1 - step / num_steps) # linear learning rate decay
    for i, p in enumerate(params):
        m[i] = beta1 * m[i] + (1 - beta1) * p.grad
        v[i] = beta2 * v[i] + (1 - beta2) * p.grad ** 2
        m_hat = m[i] / (1 - beta1 ** (step + 1))
        v_hat = v[i] / (1 - beta2 ** (step + 1))
        p.data -= lr_t * m_hat / (v_hat ** 0.5 + eps_adam)
        p.grad = 0

    # Report sparsity statistics every 100 steps
    if (step + 1) % 100 == 0:
        avg_attend = min(local_window, n) + min(top_k_global, max(0, n - local_window))
        sparsity_ratio = avg_attend / max(1, n) if n > 0 else 1.0
        print(f"step {step+1:4d} / {num_steps:4d} | loss {loss.data:.4f} | attending to ~{avg_attend:.1f}/{n} tokens ({sparsity_ratio*100:.1f}% density)")
    else:
        print(f"step {step+1:4d} / {num_steps:4d} | loss {loss.data:.4f}")



step    1 / 1000 | loss 3.4876
step    2 / 1000 | loss 3.4306
step    3 / 1000 | loss 3.3536
step    4 / 1000 | loss 3.3815
step    5 / 1000 | loss 3.3145
step    6 / 1000 | loss 3.3011
step    7 / 1000 | loss 3.2203
step    8 / 1000 | loss 3.2358
step    9 / 1000 | loss 3.0648
step   10 / 1000 | loss 3.3749
step   11 / 1000 | loss 3.0410
step   12 / 1000 | loss 3.0969
step   13 / 1000 | loss 3.2723
step   14 / 1000 | loss 3.0880
step   15 / 1000 | loss 3.0985
step   16 / 1000 | loss 2.7939
step   17 / 1000 | loss 2.9402
step   18 / 1000 | loss 3.2151
step   19 / 1000 | loss 2.8140
step   20 / 1000 | loss 2.9276
step   21 / 1000 | loss 3.1479
step   22 / 1000 | loss 3.0873
step   23 / 1000 | loss 2.9876
step   24 / 1000 | loss 2.3245
step   25 / 1000 | loss 3.7202
step   26 / 1000 | loss 2.5938
step   27 / 1000 | loss 3.4591
step   28 / 1000 | loss 2.8664
step   29 / 1000 | loss 2.4308
step   30 / 1000 | loss 2.6233
step   31 / 1000 | loss 2.9689
step   32 / 1000 | loss 2.9488
step   3

In [8]:
# Inference: may the model babble back to us
temperature = 0.5 # in (0, 1], control the "creativity" of generated text, low to high
print("\n--- inference (new, hallucinated names with DSA + RoPE) ---")
for sample_idx in range(20):
    keys, values = [[] for _ in range(n_layer)], [[] for _ in range(n_layer)]
    token_id = BOS
    sample = []
    for pos_id in range(block_size):
        logits = gpt(token_id, pos_id, keys, values)
        probs = softmax([l / temperature for l in logits])
        token_id = random.choices(range(vocab_size), weights=[p.data for p in probs])[0]
        if token_id == BOS:
            break
        sample.append(uchars[token_id])
    print(f"sample {sample_idx+1:2d}: {''.join(sample)}")


--- inference (new, hallucinated names with DSA + RoPE) ---
sample  1: non
sample  2: alya
sample  3: alilon
sample  4: jara
sample  5: ranna
sample  6: maise
sample  7: jajan
sample  8: maya
sample  9: varer
sample 10: janaya
sample 11: salan
sample 12: ayna
sample 13: sorili
sample 14: jena
sample 15: arinna
sample 16: inan
sample 17: arilelin
sample 18: biabi
sample 19: karel
sample 20: alia
